# Práctica 1. Resolución de problemas con búsqueda 
## Inteligencia Artificial I    2021/2022

La práctica está organizada en 4 partes:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 
* Parte III: aprenderemos a medir las propiedades de los algoritmos.
* Parte IV: se os dan 2 problemas algo más complejos para resolver al menos 1.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

**Fecha de entrega de la práctica completa: 18 de octubre** 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros
 
El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

In [18]:
#Librerías a usar:

from search import Problem

import numpy as np
import math

In [2]:
## Copiamos aquí la definición de la clase Problem para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

El algoritmo llama a actions, que devuelve una lista con todas las acciones aplicables. Se añaden a la lista frontera
De la lista frontera el alg escoge una, la que va a aplicar.
La acción elegida se le pasa al result junto con el estado actual

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [3]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [11]:
p =Jarras()
p.initial

(0, 0)

In [12]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [13]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [14]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

AttributeError: ignored

In [15]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [ ]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)
    """
        1. Asumir que hay infinitas barcas:
            h1(n) = (C[i] + M[i]) / 2

        2. Relajado -> los caníbales nunca se comen a los misioneros:
            h2(n) = 2*(C[i] + M[i]) - orilla(n)
    """

    def h1(self):
        return (self[0] + self[1]) / 2

    def h2(self):
        return 2*(self[0] + self[1]) - self[2]



In [ ]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [ ]:
misioneros.initial

(3, 3, 0)

In [ ]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [19]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)

    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
        if pos_hueco not in (0,3,6):
            accs.append("Mover hueco izquierda")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)

    def find_blank_square(self, state):
        """Return the index of the blank square in a given state"""
        return state.index(0)

    def goal_test(self, state):
        """ Given a state, return True if state is a goal state or False, otherwise """
        return state == self.goal

    def h(self):
        """ Return the heuristic value for a given state. """
        return 1 

        #Comprueba si el estado dado tiene solución
    def check_solvability(self, state):
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                    if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                        inversion += 1
        return inversion % 2 == 0

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [5]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [6]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [7]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [ ]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: ignored

In [ ]:
p8.result(p8.result(p8.initial,"Mover hueco arriba"),"Mover hueco abajo")

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [ ]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [ ]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [ ]:
class Quince_Puzzle(Problem):
	## Completa la definición del problema
	def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
		""" Definición de estado final 
		e inicialización del problema """
		self.goal = goal
		Problem.__init__(self, initial, goal)

	def actions(self, state):
		pos_hueco=state.index(0) # busco la posicion del 0
		actionsL=list()

		if pos_hueco not in (0, 1, 2, 3):
			actionsL.append("Mover hueco arriba")
		if pos_hueco not in (len(state) - 4,len(state) - 3, len(state) - 2, len(state) - 1):
			actionsL.append("Mover hueco abajo")
		if (pos_hueco % 4) != 0:
			actionsL.append("Mover hueco izquierda")
		if ((pos_hueco + 1) % 4) != 0:
			actionsL.append("Mover hueco derecha")
		return actionsL

	def result(self,state,action):
		pos_hueco = state.index(0)
		l = list(state)

		if action.count("ar",11) == 1:
			l[pos_hueco] = l[pos_hueco - 4]
			l[pos_hueco - 4] = 0
		if action.count("ab",11) == 1:
			l[pos_hueco] = l[pos_hueco + 4]
			l[pos_hueco + 4] = 0
		if action.count("iz",11) == 1:
			l[pos_hueco] = l[pos_hueco - 1]
			l[pos_hueco - 1] = 0
		if action.count("de",11) == 1:
			l[pos_hueco] = l[pos_hueco + 1]
			l[pos_hueco + 1] = 0

		return tuple(l)

	def h(self, node):
		return 1

		#Comprueba si el estado dado tiene solución
	def check_solvability(self, state):
		inversion = 0
		for i in range(len(state)):
			for j in range(i+1, len(state)):
				if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
					inversion += 1
		return inversion % 2 == 0
	

Representación del Problema(estado inicial Y estado objetivo)


				Table Initial:							Table Goal:
					|  1 |  2 | 3  | 4  |					|  1 |  2 | 3  | 4  |
					|  8 |  7 | 6  | 5  |					|  5 |  6 | 7  | 8  |
					|  9 | 10 | 0  | 12 |					|  9 | 10 | 11 | 12 |
					| 11 | 13 | 14 | 15 |					| 13 | 14 | 15 | 0  |


Realiza algunas pruebas para comprobar que la clase está bien definida. 

Pruebas de la clase Quince_Puzzle:

In [ ]:
p15 = Quince_Puzzle((1, 2, 3, 4, 8, 7, 6, 5, 9, 10, 14, 12, 11, 13, 0, 15))
p15.initial

(1, 2, 3, 4, 8, 7, 6, 5, 9, 10, 14, 12, 11, 13, 0, 15)

In [ ]:
p15.actions(p15.initial)

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [ ]:
p15.result(p15.initial,"Mover hueco arriba")

NameError: ignored

In [ ]:
p15.result(p15.initial,"Mover hueco abajo")

IndexError: list index out of range

In [ ]:
p15.result(p15.initial,"Mover hueco derecha")

(1, 2, 3, 4, 8, 7, 6, 5, 9, 10, 14, 12, 11, 13, 15, 0)

In [ ]:
p15.result(p15.result(p15.initial,"Mover hueco derecha"),"Mover hueco derecha")

IndexError: list index out of range

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [20]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [21]:
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima-python/')
import search

In [ ]:
## comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.  

In [16]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [7]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [ ]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

# breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

In [9]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [ ]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si el método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## ...............................................................................

Los algoritmos completos garantizan llegar al nivel donde se encuentra la solución, si la hay.

Por tanto, al tener una llamada que no funciona deducimos que el estado inicial no tiene solución.
De aquí en adelante comprobaremos la solucionidad del estado inicial antes de realizar la llamada.

In [5]:
# Añade a la definición del problema Ocho_Puzzle la función solvability que comprueba si un estado inicial tiene solución.
def check_solvability(self, state):
        """ Checks if the given state is solvable """
# The solvability of a configuration can be checked by calculating the Inversion Permutation. 
#If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0        

In [22]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

sol

'Este estado inicial no tiene solucion'

In [19]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [23]:
state8 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [32]:
%%timeit
breadth_first_tree_search(state8).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

7.56 ms ± 12.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


TODO No me da la RAM

In [30]:
#%%timeit
depth_first_graph_search(state8).solution()

KeyboardInterrupt: 

In [25]:
%%timeit
breadth_first_graph_search(state8).solution()

4.26 ms ± 31.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: 
- Puzle de 8:


**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [33]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
#Puedes utilizar alguna de las heurísticas vistas en clase. 
#Formato: misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))  --- (misioneros,canibales,orilla)

# Heurísticas para el problema de los misioneros.
"""
1. Asumir que hay infinitas barcas:
    h1(n) = (C[i] + M[i]) / 2

2. Relajado -> los caníbales nunca se comen a los misioneros:
    h2(n) = 2*(C[i] + M[i]) - orilla(n)
"""

def h1(node):
    return (node[0] + node[1]) / 2

def h2(node):
    return 2*(node[0] + node[1]) - node[2]

In [34]:
#### 4.2. Para el problema del puzle de 8 y de 16 se pide definir  las siguientes funciones heurísticas:
""" 
* linear(node): cuenta el número de casillas mal colocadas respecto al estado final.
* manhattan(node): suma la distancia Manhattan desde cada casilla a la posición en la que debería estar en el estado final.
* max_heuristic(node: maximo de las dos anteriores
* sqrt_manhattan(node):  raíz cuadrada de la distancia Manhattan
"""

# Heuristicas para el 8 Puzzle. Puedes definir las funciones fuera de la clase ya que en la llamada a A* puedes pasar el nombre 
# de la función. 

def linear(node):
    state = node.state
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    count = 0
    
    for i in range(len(state)):
        if state[i] != goal[i]:
            count += 1
    
    return count


def manhattan(node):
    state = node.state
    #goal = node.state.goal
    goal = (1,2,3,4,5,6,7,8,0)
    mhd = 0
    
    for i in range(len(state)):
        aux = goal.index(state[i])
        mhd += abs(aux//3 - i/3) + abs(aux%3 - i%3)

    return mhd
                
def sqrt_manhattan(node):
    #state = node.state
    return math.sqrt(manhattan(node))

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)
        


### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [40]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

TypeError: h() takes 1 positional argument but 2 were given

In [41]:
astar_search(puzle,manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [ ]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [42]:
astar_search(puzle,linear).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [43]:
astar_search(puzle,max_heuristic).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [44]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [ ]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

10000 loops, best of 5: 137 µs per loop


In [45]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

2.45 ms ± 5.32 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [46]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

45.3 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [47]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

2.64 ms ± 6.14 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


####  Realiza pruebas también con el puzle de 16 con los estados iniciales que quieras.  Escribe aquí tus conclusiones sobre qué heurística te parece mejor para este problema.  

Ten en cuenta que el número de posibles estados iniciales es n!, siendo n el número de fichas (números y hueco). Por tanto, en el puzle 4x4 (16 fichas), tendremos más de 130.000 millones de posibles estados iniciales. Sin embargo, sólo la mitad de esas combinaciones tiene solución. En el caso del siguiente estado inicial que sí tiene solución: (1,2,3,4,5,6,7,8,9,10,12,15,13,14,11,0) indica los nodos generados por A* con alguna de las heurísticas y comparalo con el puzle de 8.


#### Justifica claramente la respuesta basándote en los resultados esperados  (según las clases teóricas) y los resultados obtenidos en las pruebas anteriores.

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------


## Parte III:  Calcular estadísticas sobre la ejecución de los algoritmos para resolución de problemas de ocho puzzle. 
### El objetivo es comprobar experimentalmente las propiedades teóricas de los algoritmos vistas en clase.
Usaremos la función %timeit para medir los tiempos y para el espacio una version modificada de Problema que almacena el número de nodos.



En la clase modificada también vamos a incluir el calculo que nos diga si un cierto tablero tiene solución o no. Esto es muy útil.. como hemos comentado al principio solo algunos tableros tienen solucion. En el caso del puzle de 8 un tablero se ha demostrado que se puede comprobar calculando su paridad (o número de inversiones). Si es impar el tablero **no** tiene solución.  El concepto de inversión de una ficha será la suma del número de fichas que se encuentran en una posición superior a dicha ficha y que deberían estar situadas en una posición inferior. La inversión total será la suma de las inversiones individuales. Si este número es par, el puzzle tendrá solución. En caso contrario, no habrá solución. 

In [49]:
# Hacemos una definición ampliada de la clase Problem de AIMA que nos va a permitir experimentar con distintos
# estados iniciales, algoritmos y heurísticas. 

# Añadimos en la clase ampliada la capacidad para contar el número de nodos analizados durante la
# búsqueda:


class Problema_con_Analizados(Problem):

    """Es un problema que se comporta exactamente igual que el que recibe al
       inicializarse, y además incorpora unos atributos nuevos para almacenar el
       número de nodos analizados durante la búsqueda. De esta manera, no
       tenemos que modificar el código del algoritmo de búsqueda.""" 
         
    def __init__(self, problem):
        self.initial = problem.initial
        self.problem = problem
        self.analizados  = 0
        self.goal = problem.goal

    def actions(self, estado):
        return self.problem.actions(estado)

    def result(self, estado, accion):
        return self.problem.result(estado, accion)

    def goal_test(self, estado):
        self.analizados += 1
        return self.problem.goal_test(estado)

    def coste_de_aplicar_accion(self, estado, accion):
        return self.problem.coste_de_aplicar_accion(estado,accion)

In [50]:
estado_inicial = (1,2,3,4,5,6,7,0,8)
p8p=Problema_con_Analizados(Ocho_Puzzle(estado_inicial))
p8 = Ocho_Puzzle(estado_inicial)

In [51]:
p8p.initial

(1, 2, 3, 4, 5, 6, 7, 0, 8)

In [52]:
p8p.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [53]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
astar_search(puzzle_1, manhattan).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [54]:
astar_search(p8, manhattan).solution()

['Mover hueco derecha']

In [55]:
astar_search(p8p, manhattan).solution()

['Mover hueco derecha']

In [56]:
def resuelve_ocho_puzzle(estado_inicial, algoritmo, h=None):
    """Función para aplicar un algoritmo de búsqueda dado al problema del ocho
       puzzle, con un estado inicial dado y (cuando el algoritmo lo necesite)
       una heurística dada.
       Ejemplo de uso:
           puzzle_1 = (2, 4, 3, 1, 5, 6, 7, 8, 0)
           resuelve_ocho_puzzle(puzzle_1,astar_search,h2_ocho_puzzle)
        Solución: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco arriba', 
        'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo']
        Algoritmo: astar_search
        Heurística: h2_ocho_puzzle
        Longitud de la solución: 8. Nodos analizados: 11
       """
    p8=Ocho_Puzzle(estado_inicial)
    p8p=Problema_con_Analizados(p8)
    if p8.check_solvability(estado_inicial):
        if h: 
            sol= algoritmo(p8p,h).solution()
        else: 
            sol= algoritmo(p8p).solution()
        print("Solución: {0}".format(sol))
        print("Algoritmo: {0}".format(algoritmo.__name__))
        if h: 
            print("Heurística: {0}".format(h.__name__))
        else:
            pass
        print("Longitud de la solución: {0}. Nodos analizados: {1}".format(len(sol),p8p.analizados))
    else: 
        print("Este problema no tiene solucion. ")


In [57]:
resuelve_ocho_puzzle(estado_inicial,astar_search,manhattan)

Solución: ['Mover hueco derecha']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 1. Nodos analizados: 2


In [58]:
##BORRAR ESTO EN EL ENUNCIADO
E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)
#resuelve_ocho_puzzle(E5,astar_search,manhattan)

### Ejercicio 6:  resolver usando las distintas búsquedas y en su caso, las distintas heurísticas, el problema del 8 puzzle para los siguientes estados iniciales:

E1 = (2,7,3,4,6,8,1,0,5)
E2 = (3,0,1,8,4,6,5,2,7)
E3 = (4,5,6,1,0,3,7,8,2)
E4 = (1,2,3,0,5,6,4,7,8)
E5 = (0,8,7,6,5,4,3,2,1)

          E1              E2              E3              E4               E5
           
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+  
     | 2 | 7 | 3 |   | 3 | 0 | 1 |   | 4 | 5 | 6 |   | 1 | 2 | 3 |    | H | 8 | 7 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 4 | 6 | 8 |   | 8 | 4 | 6 |   | 1 | 0 | 3 |   | H | 5 | 6 |    | 6 | 5 | 4 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
     | 1 | H | 5 |   | 5 | 2 | 7 |   | 7 | 8 | 2 |   | 4 | 7 | 8 |    | 3 | 2 | 1 |
     +---+---+---+   +---+---+---+   +---+---+---+   +---+---+---+    +---+---+---+
   
 Se pide, en cada caso, obtener detalles del tiempo y espacio necesario para la resolución de estos estados.
 Hacerlo con la función resuelve_ocho_puzzle, para
 obtener, además de la solución, la longitud (el coste) de la solución
 obtenida y el número de nodos analizados. Anotar los resultados en la siguiente tabla (L, longitud de la solución, NA, nodos analizados, T, tiempo)
 
-----------------------------------------------------------------------------------------
                                       E1           E2           E3          E4          E5
    Anchura                            L= 17        L= 0         L= 20       L= 3        L= 28
                                       T= 7.1 s     T= 0 s       T= 54.7 s   T= 0  ns    T= 6min 23s
                                       NA= 19773    NA= 0        NA= 57057   NA= 13      NA= 178224
   
    Profundidad                        L= 55689     L= 0         L= 39720    L= 27       L= 64830
                                       T= 5min 23s  T= 0 s       T= 2min 11s T= 1 ms     T= 12min 13s
                                       NA= 65442    NA= 0        NA= 42926   NA= 28      NA= 106784
                                       
    Coste Uniforme                     L= 17        L= 0         L= 20       L= 3        L= 28
                                       T= 23,4 s    T= 0         T= 3min 54s T= 0 ns     T= 27min 6s
                                       NA= 14105    NA= 0        NA= 48428   NA= 13      NA= 177480
                                       
    Primero el mejor (linear)          L= 43        L= 0         L= 76       L= 3        L= 28
                                       T= 62.1 ms   T= 0         T= 70.1 ms  T= 0 ns     T= 1 ms
                                       NA= 689      NA= 0        NA= 748     NA= 4       NA= 29
                                                                            
    Primero el mejor (manhattan)       L= 85        L= 0         L= 82       L= 3        L= 84
                                       T= 98.1 ms   T= 0         T= 52 ms    T= 1e+03 µs T= 12 ms
                                       NA= 829      NA= 0        NA= 615     NA= 4       NA= 276
                                                                             
    A* (linear)                        L= 17        L= 0         L= 20       L= 3        L=
                                       T= 56.1 ms   T= 0         T= 1.37 s   T= 0 ns     T=
                                       NA= 660      NA= 0        NA= 3377    NA= 4       NA= 
                                                                                                                    
    A* (manhattan)                     L= 17        L= 0         L= 20       L= 3        L= 28
                                       T= 28 ms     T= 0         T= 512 ms   T= 1 ns     T= 7.38 s
                                       NA= 418      NA= 0        NA= 1938    NA= 4       NA= 7319


 -----------------------------------------------------------------------------------------
PUEDES AMPLIAR LA TABLA CON PRUEBAS ADICIONALES. 

Nota: E2 es un estado inicial que no tiene solución.

 **justificar los resultados con las distintas propiedades teóricas estudiadas en clase**.  



In [96]:
%%time
# Anchura
resuelve_ocho_puzzle(E5,breadth_first_graph_search)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: breadth_first_graph_search
Longitud de la solución: 28. Nodos analizados: 178224
Wall time: 6min 23s


In [97]:

%%time
# Profundidad
resuelve_ocho_puzzle(E5,depth_first_graph_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arri

In [98]:
%%time
# Coste Uniforme
resuelve_ocho_puzzle(E5,uniform_cost_search)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: uniform_cost_search
Longitud de la solución: 28. Nodos analizados: 177480
Wall time: 27min 6s


In [99]:
%%time
# Primero el mejor, linear
resuelve_ocho_puzzle(E5,best_first_graph_search,linear)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha']
Algoritmo: best_first_graph_search
Heurística: linear
Longitud de la solución: 28. Nodos analizados: 29
Wall time: 1 ms


In [100]:
%%time
# Primero el mejor, manhattan
resuelve_ocho_puzzle(E5,best_first_graph_search,manhattan)

Solución: ['Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco a

In [ ]:
%%time
# A*, linear
resuelve_ocho_puzzle(E5,astar_search,linear)

In [103]:
%%time
# A*, manhattan
resuelve_ocho_puzzle(E5,astar_search,manhattan)

Solución: ['Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo', 'Mover hueco izquierda', 'Mover hueco izquierda', 'Mover hueco arriba', 'Mover hueco arriba', 'Mover hueco derecha', 'Mover hueco derecha', 'Mover hueco abajo', 'Mover hueco abajo']
Algoritmo: astar_search
Heurística: manhattan
Longitud de la solución: 28. Nodos analizados: 7319
Wall time: 7.38 s


###  PARTE IV:  Se os proporcionará un enunciado de algún problema para representar y resolver usando el paradigma del espacio de estados. 


# Tips de teoría #

- _Graph Search_ es óptima con A* sólo si la heurística es _consistente_.
- _Tree Search_ es óptima con A* sólo si la heurística es _admisible_.